# Automated ML

In [1]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

We start by setting up our experiment in our workspace.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'AutoMLRun'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RNG8E4NPM to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-138676
Azure region: southcentralus
Subscription id: aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee
Resource group: aml-quickstarts-138676


## Dataset

The dataset consists of 70 000 records of patients data in 12 features, such as age, gender, systolic blood pressure, diastolic blood pressure, and etc. The target class "cardio" equals to 1, when patient has cardiovascular desease, and it's 0, if patient is healthy.

The task is to predict the presence or absence of cardiovascular disease (CVD) using the patient examination results.

**Data description**

There are 3 types of input features:

Objective:   *factual information*

Examination: *results of medical examination*

Subjective:  *information given by the patient*



We download the data from the github repository (https://raw.githubusercontent.com/MonishkaDas/nd00333-capstone/master/starter_file/cardio_train.csv) using TabularDatasetFactory from Azure.
 We then convert it to pandas dataframe and preprocess it and clean  it. The data consisted 
a few outliers and invalid records(like diastolic pressure,ap_lo was higher than systolic pressure,ap_hi in a few cases). 
We standardize a few columns (["age", "height", "weight", "ap_hi", "ap_lo"])
 and get dummy values for categorical columns (["cholestrol","gluc"]).

In [3]:
web_path = "https://raw.githubusercontent.com/MonishkaDas/nd00333-capstone/master/starter_file/cardio_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path=web_path,separator=";")

df=ds.to_pandas_dataframe()


In [4]:
df.drop("id", axis = 1, inplace = True)

s_list = ["age", "height", "weight", "ap_hi", "ap_lo"]

def standartization(x):
    x_std = x.copy(deep=True)
    for column in s_list:
        x_std[column] = (x_std[column]-x_std[column].mean())/x_std[column].std()
    return x_std 
df=standartization(df)

In [5]:
df.gender = df.gender.replace(2,0)

df.drop(df[(df['height'] > df['height'].quantile(0.975)) | (df['height'] < df['height'].quantile(0.025))].index,inplace=True)
df.drop(df[(df['weight'] > df['weight'].quantile(0.975)) | (df['weight'] < df['weight'].quantile(0.025))].index,inplace=True)

df.drop(df[(df['ap_hi'] > df['ap_hi'].quantile(0.975)) | (df['ap_hi'] < df['ap_hi'].quantile(0.025))].index,inplace=True)
df.drop(df[(df['ap_lo'] > df['ap_lo'].quantile(0.975)) | (df['ap_lo'] < df['ap_lo'].quantile(0.025))].index,inplace=True)

duplicated = df[df.duplicated(keep=False)]
duplicated = duplicated.sort_values(by=['age', "gender", "height"], ascending= False)
df.drop_duplicates(inplace=True)

df['cholesterol']=df['cholesterol'].map({ 1: 'normal', 2: 'above normal', 3: 'well above normal'})
df['gluc']=df['gluc'].map({ 1: 'normal', 2: 'above normal', 3: 'well above normal'})
dummies = pd.get_dummies(df[['cholesterol','gluc']],drop_first=True)
df = pd.concat([df,dummies],axis=1)
df.drop(['cholesterol','gluc'],axis=1,inplace=True)

df.head()

,age,gender,height,weight,ap_hi,ap_lo,smoke,alco,active,cardio,cholesterol_normal,cholesterol_well above normal,gluc_normal,gluc_well above normal
0,-0.436058,0,0.443449,-0.847867,-0.122181,-0.088238,0,0,1,0,1,0,1,0
1,0.307684,1,-1.018161,0.749826,0.072610,-0.035180,0,0,1,1,0,1,1,0
2,-0.247995,1,0.078046,-0.708937,0.007679,-0.141296,0,0,0,1,0,1,1,0
3,-0.748147,0,0.565250,0.541431,0.137540,0.017878,0,0,1,1,1,0,1,0
4,-0.808538,1,-1.018161,-1.264657,-0.187111,-0.194354,0,0,0,0,1,0,1,0


Now we store our processed dataset in the datastore so it can be retrieved using TabularDatasetFactory and then passed to the configured automl model

In [6]:
outname='Cardiovascular-Dataset.csv'
outdir='dataset/'
if not os.path.exists(outdir):
    os.mkdir(outdir)

fullpath=os.path.join(outdir,outname)
df.to_csv(fullpath)

We store our dataset in our default datastore in order to access it.

In [7]:
datastore = ws.get_default_datastore()

In [8]:
datastore.upload(src_dir = "dataset/", target_path = "data/")

Uploading an estimated of 1 files
Uploading dataset/Cardiovascular-Dataset.csv
Uploaded dataset/Cardiovascular-Dataset.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_765bd6c1bb9c44618b0a51543c0270d0

In [11]:
training_data = TabularDatasetFactory.from_delimited_files(path = [(datastore, ("data/Cardiovascular-Dataset.csv"))])

## AutoML Configuration

We start by setting up our compute cluster, where we will run our automl run.

In [12]:
cpu_cluster_name = "compute-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D3_V2',
                                                           max_nodes=10)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


We use the following configuration for our run


| Configuration | Reason |
| :- | :- |
| **experiment_timeout_minutes** | Maximum time that all iterations combined can take before the experiment terminates. |
|**max_concurrent_iterations**|These are the maximumm iterations occuring simultaneously, in this case the value is set as 10|
|**n_cross_validations**|We use 5 cross validations to avoid overfitting |
|**primary_metric**| The primary metric for this experiment is Accuracy|
|**task**|Classification |
|**compute_target**|This is the compute cluster we will be using for the run |
|**training_data**|This is the training dataset derived from the default datastore  |
|**label_column_name**|This is the target column, in this case **"cardio"**|
|**enable_onnx_compatible_model**|This is set as **True** to make the model onnx compatible|



In [13]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes" :30,
    "max_concurrent_iterations": 10,
    "n_cross_validations": 5,
    "primary_metric": 'accuracy',
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task="classification",
    compute_target=cpu_cluster,
    training_data=training_data,
    label_column_name="cardio",
    enable_onnx_compatible_models=True,
    **automl_settings
    )

After setting up the configurations of the Automl experiment, we submit the run

In [14]:
remote_run = experiment.submit(config = automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on compute-cluster with default configuration
Running on remote compute: compute-cluster
Parent Run ID: AutoML_d2d0cbc7-8762-4eaf-9bb3-1cd440986de7

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing v

## Run Details
The `Rundetails` widget, as the name suggests gives us greater insight about how the Run is proceeding, enabling us to monitor and understand the situation, thereby dealing with it accordingly.

In [25]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_d2d0cbc7-8762-4eaf-9bb3-1cd440986de7',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-02-12T10:07:10.658192Z',
 'endTimeUtc': '2021-02-12T10:52:00.7911Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoMLRun","subscription_id":"aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee","resource_group":"aml-quickstarts-138676","workspace_name":"quick-starts-ws-138676","region":"southcentralus","compute_target":"compute-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":n

## Best Model

The best performing model is the `VotingEnsemble`

**Accuracy :  0.73125**. 

**average_precision_score_weighted : ** 0.78  **f1_score_weighted :** 0.73

**best_individual_pipeline_score :**  0.7295652159597188

**ensembled_algorithms :**  ['LightGBM', 'XGBoostClassifier', 'XGBoostClassifier', 'LightGBM', 'LightGBM', 'XGBoostClassifier', 'XGBoostClassifier', 'XGBoostClassifier', 'LightGBM', 'XGBoostClassifier']

**ensemble_weights :** [0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693, 0.3076923076923077, 0.07692307692307693, 0.07692307692307693, 0.07692307692307693]



In [21]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

model_ml = best_run.register_model(model_name='Cardio-AutoMl', model_path='./')

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Run(Experiment: AutoMLRun,
Id: AutoML_d2d0cbc7-8762-4eaf-9bb3-1cd440986de7_205,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               scale_pos_weight=1,
                                                                                               seed=None,
                                               

## Model Deployment

Being the better performing model, I shall hereby deploy the `VotingEnsemble` model.

In [28]:
from azureml.core.model import Model
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

In [29]:
os.makedirs('./amlmodel', exist_ok=True)

best_run.download_file('/outputs/model.pkl',os.path.join('./amlmodel','Cardio_Automl_best_model.pkl'))

for f in best_run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join('./amlmodel', f.split('/')[-1])
        print(f'Downloading from {f} to {output_file_path} ...')
        best_run.download_file(name=f, output_file_path=output_file_path)


TODO: In the cell below, send a request to the web service you deployed to test it.

In [30]:
model=best_run.register_model(
            model_name = 'cardio-automl-bestmodel', 
            model_path = './outputs/model.pkl',
            model_framework=Model.Framework.SCIKITLEARN,
            description='Cardiovascular Disease Prediction - Best Model'
)

In [32]:
# Download the conda environment file and define the environement
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'conda_env.yml')
myenv = Environment.from_conda_specification(name = 'myenv',
                                             file_path = 'conda_env.yml')

In [33]:
# download the scoring file produced by AutoML
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score_cardio.py')

# set inference config
inference_config = InferenceConfig(entry_script= 'score_cardio.py',
                                    environment=myenv)

In [34]:
# set Aci Webservice config
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True)

In [35]:
service = Model.deploy(workspace=ws, 
                       name='cardio-bestmodel', 
                       models=[model], 
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

In [36]:
service

AciWebservice(workspace=Workspace.create(name='quick-starts-ws-138676', subscription_id='aa7cf8e8-d23f-4bce-a7b9-1f0b4e0ac8ee', resource_group='aml-quickstarts-138676'), name=cardio-bestmodel, image_id=None, compute_type=None, state=ACI, scoring_uri=Transitioning, tags=None, properties={}, created_by={})

In [37]:
# wait for deployment to finish and display the scoring uri and swagger uri
service.wait_for_deployment(show_output=True)

print('Service state:')
print(service.state)

print('Scoring URI:')
print(service.scoring_uri)

print('Swagger URI:')
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.......................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Service state:
Healthy
Scoring URI:
http://fe787e3b-f335-4d40-86e5-1f3efaefa182.southcentralus.azurecontainer.io/score
Swagger URI:
http://fe787e3b-f335-4d40-86e5-1f3efaefa182.southcentralus.azurecontainer.io/swagger.json


In [39]:
!python3 logs.py

2021-02-12T12:04:40,618979987+00:00 - gunicorn/run 
2021-02-12T12:04:40,619891608+00:00 - iot-server/run 
2021-02-12T12:04:40,620022711+00:00 - rsyslog/run 
2021-02-12T12:04:40,622671970+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_7785023fceb74e4facc1b1a577b1faf9/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

Consumint the endpoint using `endpoint.py`

In [51]:
!python3 endpoint.py

{"result": [0, 1, 1]}


In [53]:
import json

# select 3  samples from the dataframe
x_df=df.sample(5)
y_df = x_df.pop('cardio')

x_df['Column1'] = 0.0

# convert the records to a json data file
recorded=x_df.to_dict(orient='records')

scoring_json = json.dumps({'data': recorded})
print(scoring_json)

{"data": [{"age": -0.9131074240207604, "gender": 1, "height": -0.4091567440347836, "weight": 1.5139398703878981, "ap_hi": 0.007679393449469267, "ap_lo": -0.03517973826363841, "smoke": 0, "alco": 0, "active": 1, "cholesterol_normal": 1, "cholesterol_well above normal": 0, "gluc_normal": 1, "gluc_well above normal": 0, "Column1": 0.0}, {"age": -0.7051837627213113, "gender": 1, "height": 0.44344889066469956, "weight": 0.40250124598518133, "ap_hi": 0.0726096436563594, "ap_lo": -0.03517973826363841, "smoke": 0, "alco": 0, "active": 0, "cholesterol_normal": 0, "cholesterol_well above normal": 0, "gluc_normal": 1, "gluc_well above normal": 0, "Column1": 0.0}, {"age": -0.4619981939500259, "gender": 1, "height": -0.7745591589059906, "weight": -0.08375315219100726, "ap_hi": -0.12218110696431099, "ap_lo": -0.08823786818990224, "smoke": 0, "alco": 0, "active": 1, "cholesterol_normal": 1, "cholesterol_well above normal": 0, "gluc_normal": 1, "gluc_well above normal": 0, "Column1": 0.0}, {"age": -1.

In [54]:
output = service.run(scoring_json)
output

'{"result": [1, 1, 0, 0, 0]}'

In [55]:
y_df

16993    1
52236    1
48160    0
52201    0
31680    0
Name: cardio, dtype: int64

Enabling logging using `logs.py`.

# Retrieve and Save ONNX Model

In [27]:
from azureml.automl.runtime.onnx_convert import OnnxConverter

b_run , onnx_mdl = remote_run.get_output(return_onnx_model=True)
onnx_fl_path = "./best_model.onnx"
OnnxConverter.save_onnx_model(onnx_mdl, onnx_fl_path)

## Predict with the ONNX model, using onnxruntime package

In [56]:
import sys
import json
from azureml.automl.core.onnx_convert import OnnxConvertConstants
from azureml.train.automl import constants

df_train, df_test = train_test_split(df, test_size=0.01)

if sys.version_info < OnnxConvertConstants.OnnxIncompatiblePythonVersion:
    python_version_compatible = True
else:
    python_version_compatible = False

import onnxruntime
from azureml.automl.runtime.onnx_convert import OnnxInferenceHelper

def get_onnx_res(run):
    res_path = 'onnx_resource.json'
    run.download_file(name=constants.MODEL_RESOURCE_PATH_ONNX, output_file_path=res_path)
    with open(res_path) as f:
        onnx_res = json.load(f)
    return onnx_res

if python_version_compatible:
    mdl_bytes = onnx_mdl.SerializeToString()
    onnx_res = get_onnx_res(b_run)

    df_test['Column1'] = 0.0
    onnxrt_helper = OnnxInferenceHelper(mdl_bytes, onnx_res)
    pred_onnx, pred_prob_onnx = onnxrt_helper.predict(df_test)

    print(pred_onnx)
    print(pred_prob_onnx)
else:
    print('Please use Python version 3.6 or 3.7 to run the inference helper.')

[0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1
 0 0 0 0 1 0 0 0 1 0 0 1 0 1 1 0 0 0 1 0 0 0 1 1 0 0 1 0 0 1 1 0 0 1 0 1 1
 0 1 0 0 1 0 1 0 0 0 1 1 0 1 0 0 1 1 0 0 1 0 0 0 1 0 0 1 0 1 1 0 1 0 0 0 0
 0 1 1 0 0 0 0 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 1 0 1 0 1 0 0 0 0 0 1 0 1 1
 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 1 1 1 0
 0 1 0 1 0 0 0 0 0 1 1 1 1 0 1 0 1 1 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 1 1 0
 0 0 0 0 1 0 0 0 1 1 1 1 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 1 0 1 0 0
 0 1 0 1 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 1 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 1 0 1 1 0 1 1 1 1 0 1 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 1 1 1 0 1 0 1 0 1 0 0
 1 0 0 1 1 0 0 0 0 0 1 0 0 1 1 0 0 0 1 0 0 0 1 0 1 1 1 0 0 0 0 1 1 1 1 0 0
 0 0 1 0 0 1 1 0 0 1 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 1 1 0 0 1 1 0 1 0 0 0 0
 1 0 1 1 1 1 0 1 0 0 0 1 0 1 0 0 1 1 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0
 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 0 1 0 0 0 0 0 1 1 1 0 1
 0 1 1 0 0 1 0 0 1 1 0 0 

In [ ]:
service.delete()